In [2]:
import os
os.environ['HSA_OVERRIDE_GFX_VERSION'] = '10.3.0'

In [3]:
import torch

In [4]:

bs = 10
target = torch.rand((bs,1),dtype=torch.float32) * 100
mae = 40
offset = torch.rand((bs,1),dtype=torch.float32) * mae
# pred = torch.rand((1,bs),dtype=torch.float32) * 10
pred = target + (offset - (offset.mean() / 2))
print(target)
print(pred)

l1Loss = torch.nn.L1Loss()
l1Loss.forward(pred,target)

tensor([[71.6880],
        [79.1629],
        [75.6734],
        [40.9135],
        [35.4407],
        [49.8642],
        [10.6525],
        [ 0.6885],
        [97.4801],
        [87.6986]])
tensor([[75.0360],
        [99.8792],
        [73.9922],
        [47.7149],
        [65.0679],
        [77.5228],
        [ 3.4557],
        [17.8531],
        [92.3823],
        [95.4117]])


tensor(12.7005)

In [14]:
sortedI = target.argsort(dim=0)
sortedI[0]
pred[sortedI[0]]

tensor([[17.8531]])

In [166]:
sortedI = target.argsort(dim=0)
print(sortedI)
sortedTarget = target[sortedI,:].squeeze(2)
sortedPred = pred[sortedI,:].squeeze(2)
# sortedTarget = target
# sortedPred = pred
sortedTarget.shape,sortedTarget, sortedPred

tensor([[2],
        [7],
        [4],
        [0],
        [8],
        [9],
        [5],
        [6],
        [1],
        [3]])


(torch.Size([10, 1]),
 tensor([[ 7.5831],
         [15.2481],
         [24.0031],
         [26.8108],
         [33.4258],
         [35.2974],
         [52.1167],
         [65.8139],
         [71.6114],
         [88.3461]]),
 tensor([[ 29.6623],
         [ 33.6221],
         [ 24.6596],
         [ 33.5735],
         [ 41.4102],
         [ 54.9278],
         [ 47.5299],
         [ 61.1063],
         [ 69.0397],
         [113.4024]]))

In [167]:
margin = 20 # <1 easy >1 hard
a = 0.4

p02 = sortedPred[0::2,:]
p13 = sortedPred[1::2,:]
t02 = sortedTarget[0::2,:]
t13 = sortedTarget[1::2,:]
t = (t02 - t13).sign()
tm = (t02 - t13).abs().mean()
print("p02", p02)
print("p13", p13)
print("t02", t02)
print("t13", t13)
print("ts", t)
print("tm", tm)

margin = tm * a
ml = torch.nn.MarginRankingLoss(margin)
ml1 = ml.forward(p02,p13, t)
print("loss",ml1)

p12 = p02[1:,:]
p23 = p13[0:-1,:]
t12 = t02[1:,:]
t23 = t13[0:-1,:]
tt = (t12 - t23).sign()
ttm = (t12 - t23).abs().mean()
margin = ttm * a
ml = torch.nn.MarginRankingLoss(margin)
print("pp02", p12)
print("pp13", p23)
print("tt02", t12)
print("tt13", t23)
print("tts", tt)
print("ttm", ttm)
ml2 = ml.forward(p12,p23, tt)
print("loss",ml2)
print(ml1 + ml2)

p02 tensor([[29.6623],
        [24.6596],
        [41.4102],
        [47.5299],
        [69.0397]])
p13 tensor([[ 33.6221],
        [ 33.5735],
        [ 54.9278],
        [ 61.1063],
        [113.4024]])
t02 tensor([[ 7.5831],
        [24.0031],
        [33.4258],
        [52.1167],
        [71.6114]])
t13 tensor([[15.2481],
        [26.8108],
        [35.2974],
        [65.8139],
        [88.3461]])
ts tensor([[-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.]])
tm tensor(8.5552)
loss tensor(0.)
pp02 tensor([[24.6596],
        [41.4102],
        [47.5299],
        [69.0397]])
pp13 tensor([[33.6221],
        [33.5735],
        [54.9278],
        [61.1063]])
tt02 tensor([[24.0031],
        [33.4258],
        [52.1167],
        [71.6114]])
tt13 tensor([[15.2481],
        [26.8108],
        [35.2974],
        [65.8139]])
tts tensor([[1.],
        [1.],
        [1.],
        [1.]])
ttm tensor(9.4967)
loss tensor(5.9894)
tensor(5.9894)


In [180]:
class DynamicMarginRankingLoss(torch.nn.Module):
    def __init__(self, alpha:float = 0, sort:bool = True):
        super().__init__()

        self.alpha = alpha
        self.sort = sort
        self.marginLoss = torch.nn.MarginRankingLoss()

    def forward(self,input:torch.Tensor, target:torch.Tensor) -> torch.Tensor:


        if(len(input) < 4):
            return torch.tensor(0)
        
        if(len(input) % 2 != 0):
            # return torch.tensor(0)
            input = input[:-1,:]
            target = target[:-1,:]
        
        sortedI = target.argsort(dim=0)
        if(self.sort):
            sortedTarget = target[sortedI,:].squeeze(2)
            sortedPred = input[sortedI,:].squeeze(2)
        else:
            sortedTarget = target
            sortedPred = input

        p02 = sortedPred[0::2,:]
        p13 = sortedPred[1::2,:]
        t02 = sortedTarget[0::2,:]
        t13 = sortedTarget[1::2,:]
        t = (t02 - t13).sign()
        tm = (t02 - t13).abs().mean()

        margin = tm * self.alpha
        self.marginLoss.margin = margin
        loss_01_12 = self.marginLoss.forward(p02,p13, t)

        p12 = p02[1:,:]
        p23 = p13[0:-1,:]
        t12 = t02[1:,:]
        t23 = t13[0:-1,:]
        tt = (t12 - t23).sign()
        ttm = (t12 - t23).abs().mean()
        margin = ttm * self.alpha
        self.marginLoss.margin = margin
        loss_12_23 = self.marginLoss.forward(p12,p23, tt)

        return loss_01_12 + loss_12_23

In [187]:

bs = 11
target = torch.rand((bs,1),dtype=torch.float32) * 100
mae = 40
offset = torch.rand((bs,1),dtype=torch.float32) * mae
# pred = torch.rand((1,bs),dtype=torch.float32) * 10
pred = target + (offset - (offset.mean() / 2))
print(len(target))
print(target)
print(pred)

l1Loss = torch.nn.L1Loss()
l1Loss.forward(pred,target)

11
tensor([[43.7459],
        [77.6764],
        [51.1396],
        [59.5549],
        [61.1553],
        [52.9799],
        [73.0600],
        [45.2424],
        [89.5174],
        [49.0565],
        [32.1831]])
tensor([[ 42.9171],
        [ 88.7756],
        [ 42.6100],
        [ 55.5207],
        [ 72.3575],
        [ 76.9580],
        [ 69.6210],
        [ 67.7207],
        [102.1103],
        [ 79.4305],
        [ 41.8411]])


tensor(12.5649)

In [188]:
dml = DynamicMarginRankingLoss(0.4)
l = dml.forward(pred,target)
l
#21.5

tensor(13.8261)

In [164]:
dml = DynamicMarginRankingLoss(0.4, False)
l = dml.forward(pred,target)
l

tensor(4.6576)